# 7. Clustering

In [ ]:
%matplotlib inline
import warnings;warnings.filterwarnings('ignore');import matplotlib.pyplot as plt;import numpy as np;import pandas as pd;import seaborn as sns;sns.set_context("notebook", font_scale=1.4);sns.set_style("whitegrid");import imp;compomics_import = imp.load_source('compomics_import', '../compomics_import.py');from IPython.core.display import HTML;css_file = '../my.css';HTML(open(css_file, "r").read())

So far we have seen learning algorithms for **supervised data analysis**, i.e. the label or target was known for each data point. However, as labeling data can be hard and/or costly, most data that is collected is unlabeled. Learning from unlabeled data is known as **unsupervised learning** in which we look for hidden **structure** in the data that can learn us something about the true model(s) that generated the data. 

**Clustering** is a class of unsupervised learning methods that associates observations according to some specified **measure of similarity** (e.g. Euclidean distance). In its basic form (many variations exist) it tries to cluster the unlabeled data points into groups such that data points in the same group (**cluster**) are more similar to each other than those in other groups. Typical applications of clustering in bioinformatics include finding genes that co-express, proteins that interact, finding sub-populations in disease types, and image analysis tasks such as the segmentation of raw medical image data. Two popular algorithms for unsupervised data clustering are K-means and hierarchical agglomerative clustering.

The **K-means clustering algorithm** associates each data point $x^{(i)}$ in a data set ($i=1 \ldots n$) to one of $K$ clusters. Each cluster is specified by a **centroid** that is the average location of all the points in the cluster. The algorithm proceeds iteratively from arbitrary centroid locations, updating the membership of each point according to minimum distance and then updating the centroid location based on the new cluster membership. The algorithm converges when the assignment of points to centroids does not change anymore with each iteration. 

We can formalize this as follows:



1. Initialize cluster centroids (e.g. *at random*):

    $$\mu^{(0)}_1, \ldots, \mu^{(0)}_k \in \mathbb{R}^n.$$

2. Iterate steps ($s=0$) until converged:

    a. for each data point $x^{(i)}$ find the closest centroid 
    
    $$c_i = \underset{j}{\operatorname{argmax}} || x^{(i)} - \mu_j^{(s)}||,$$
    
    b. update the centroids as
    
    $$\mu_j^{(s+1)} = \frac{\sum_{i=1}^n I[c_i = j] x^{(i)}}{\sum_{i=1}^n I[c_i = j]},$$
    
    where $I[c_i = j]$ equals 1 if $c_i$ equals $j$ and 0 otherwise. 
    
    Let's show how this works on an artificial data set with four clusters in two dimensions: 

In [ ]:
dataset_blops = pd.read_csv("dataset_blops.csv")

sns.lmplot(x="x1", y="x2", hue='y', data=dataset_blops, fit_reg=False, height=6, scatter_kws={"s": 80})
plt.show()

For this data set we know that there are 4 clusters indicated by the target column '$y$'. In real unlabeled data sets this information would not be available. First we initialize the 4 cluster centroids randomly:

In [ ]:
np.random.seed(1)
start_centroids = 3*(np.random.rand(4,2)-0.5)

We can then plot these centroids:

In [ ]:
sns.lmplot(x="x1", y="x2", data=dataset_blops, fit_reg=False, height=6, scatter_kws={"s": 80})
plt.scatter(*start_centroids.T, c='k', marker='o', s=200)
plt.show()

Now we have to compute the distance between the centroids and the data points ($|| x^{(i)} - \mu_j^{(s)} ||$) to find the closest centroid $\mu_j^{(s)}$ for each data point $x^{(i)}$:

$$\text{argmin}_j || x^{(i)} - \mu_j^{(s)} ||$$


We can use the function `cdist` from the python SciPy package to calculate the distances from each point to each centroid:

In [ ]:
from scipy.spatial.distance import cdist

distances = cdist(start_centroids, dataset_blops[['x1','x2']])
print(distances)

Now, for each column $i$ in *distances* we can find the row $j$ for which the value of that column is smallest. So for each data point $x^{(i)}$ we assigned the row number of the closest centroid $\mu_j^{(s)}$:

In [ ]:
labels = distances.argmin(axis=0)
print(labels)

We now associated each data point with a cluster centroid.

Let's plot this:

In [ ]:
dataset_blops['cluster_labels'] = labels

sns.lmplot(x="x1", y="x2", data=dataset_blops, hue='cluster_labels', fit_reg=False, height=6, scatter_kws={"s": 80})
plt.scatter(*start_centroids.T, c='k', marker='o', s=200)
plt.show()

Now we can re-assign the centroid locations based on the means of the current members' locations:

$$\mu_j^{(s+1)} = \frac{\sum_{i=1}^n I[c_i = j] x^{(i)}}{\sum_{i=1}^n I[c_i = j]}.$$

In [ ]:
new_centroids = dataset_blops[['x1','x2','cluster_labels']].groupby('cluster_labels').mean().values
print(new_centroids)

In [ ]:
sns.lmplot(x="x1", y="x2", data=dataset_blops, hue='cluster_labels', fit_reg=False, height=6, scatter_kws={"s": 80})
plt.scatter(*new_centroids.T, c='k', marker='o', s=200)
plt.show()

Now we iterate these steps until convergence:

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=4,max_iter=10,n_init=1,init=start_centroids)
km.fit(dataset_blops[['x1','x2']])

dataset_blops['cluster_labels'] = km.labels_

sns.lmplot(x="x1", y="x2", data=dataset_blops, hue='cluster_labels', fit_reg=False, height=6, scatter_kws={"s": 80})
plt.scatter(*km.cluster_centers_.T, c='k', marker='o', s=200)
plt.show()

Let's do the exercise again with a different random initiation of the centroids (random seed is set to 5 instead of 1). We can see that this time the K-means algorithm fails to converge correctly.

The **K-means++ algorithm** works the same as K-means but computes a smarter initialization of the centroids. The intuition behind this approach is that spreading out the $k$ initial centroids is a good thing: the first centroid is chosen uniformly at random from the data points that are being clustered, after which each subsequent centroid is chosen from the remaining data points with probability proportional to its squared distance from the point's closest existing centroid.

The K-means algorithm is very sensitive to the scale of the features. In the example below we create an artificial data set in which the scale of feature $x_1$ is much larger than the scale of feature $x_2$:

In [ ]:
dataset_blops_scale = pd.read_csv("dataset_blops_scale.csv")

km = KMeans(n_clusters=4,max_iter=10)
km.fit(dataset_blops_scale[['x1','x2']])

dataset_blops_scale['cluster_labels'] = km.labels_

sns.lmplot(x="x1", y="x2", data=dataset_blops_scale, hue='cluster_labels', fit_reg=False, height=6, scatter_kws={"s": 80})
plt.scatter(*km.cluster_centers_.T, c='k', marker='o', s=200)
plt.show()

We observe how K-means++ fails to find the correct clusters. Scaling the features before applying K-means is highly recommended.

In real clustering tasks it is typically not known how many data clusters to expect. In this case we have to choose the number of clusters $k$ based on some criteria. One of those is the **silhouette coefficient** that measures how close various clusters are. A higher silhouette score is better as it means that we don't have too many overlapping clusters.

To compute the silhouette score we first compute two data point properties: **cohesion** and **separation**. The cohesion $a(x)$ of a data point $x$ measures how closely related the points in a cluster are. It is computed as the mean distance between the data point and all other points in the same cluster. Separation measures how well-separated a cluster is from the other features. The separation $b(x)$ of a data point $x$ is computed as the mean distance between the data point and all other points in the next nearest cluster.

The silhouette score $s(x)$ for a data point $x$ is computed as 

$$s(x)=\frac{b(x)-a(x)}{max(a(x),b(x))}.$$

The silhouette coefficient $SC$ for a given clustering of the data points is then computed as 

$$SC = \frac{1}{n}\sum\limits_{i=1}^ns(x).$$

The quality of a clustering can also be measured by the **inertia**. This metric is used to estimate how close the data points in a cluster are (a bit like cohesion discussed above). It is calculated as the sum of squared distance for each point to it's closest centroid, i.e., its assigned cluster center. The intuition behind inertia is that clusters with lower inertia are better as it means closely related points form a cluster. 

In [ ]:
from sklearn import metrics

X = dataset_blops[['x1','x2']].values

clf = KMeans(init='k-means++')

s_sil = []
s_in = []
for k in range(2,16):
    clf.n_clusters = k
    clf.fit(X)
    labels = clf.labels_
    s_sil.append(metrics.silhouette_score(X, labels))
    s_in.append(clf.inertia_)

cluster_evaluation = pd.DataFrame()
cluster_evaluation['k'] = range(2,16)
cluster_evaluation['silhouette'] = s_sil
cluster_evaluation['inertia'] = s_in
    
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
sns.pointplot(x="k", y="silhouette", data=cluster_evaluation)
plt.subplot(1,2,2)
sns.pointplot(x="k", y="inertia", data=cluster_evaluation)
plt.show()

Hierarchical **agglomerative** clustering is an algorithm that produces a clustered visualization of a data set without having to set the number of cluster $k$ in advance. The idea is to start with a singleton cluster, and clusters are iteratively merged until one single cluster remains. This results in a "cluster tree," which is also called a **dendogram**.

The algorithm can be summarized as follows:

1. represent each data point as a singleton cluster (a cluster with just one data point)
2. merge the two closest clusters 
3. repeat step 2. until one single cluster remains

So what does that mean *merge the two closest clusters*? In general we consider three possibilities (the best method depends on the data set and as such this choice can be considered a hyperparameter of hierarchical agglomerative clustering).

**Complete linkage** compares the most dissimilar members between clusters in each iteration (step 2.). The two clusters that have the most similar *dissimilar members* are merged into a new cluster. We can write this as 

$$d(A,B) = \max(dist(A_i, B_j))$$

where $A_i$ ranges over all data points in cluster $A$ and $B_j$ ranges over all data points in cluster $B$.
 
**Single linkage** compares the two most similar members between clusters in each iteration:

$$d(A,B) = \min(dist(A_i, B_j)).$$

**Average linkage** computes the mean distance between the members of each cluster:

$$d(A,B) = \frac{1}{|A||B|}\sum_{A_i \in A}\sum_{B_j \in B}dist(A_i,B_j).$$

Let's consider the following data set:

In [ ]:
dataset_hc = pd.read_csv("dataset_hc.csv")
compomics_import.plot_scatter_annotated(dataset_hc,range(len(dataset_hc)))
plt.show()

We can plot the pairwise distances between the data points as a distance matrix: 

In [ ]:
from scipy.spatial.distance import pdist, squareform

distxy = squareform(pdist(dataset_hc, metric='euclidean'))

sns.heatmap(distxy, annot=True, linewidths=.5)
plt.show()

We can then use this distance matrix to compute a cluster dendogram:

In [ ]:
# perform clustering and plot the dendrogram
from scipy.cluster.hierarchy import linkage, dendrogram

R = dendrogram(linkage(distxy, method='complete'))

In hierarchical agglomerative clustering both the rows and the columns in the data set can be clustered simultaneously. In the python seaborn package there is a function called `clustermap()` that performs hierarchical clustering of a data matrix:

In [ ]:
cg = sns.clustermap(dataset_hc,method="complete",figsize=(6,10))
plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.show()

This is a typical result of a hierarchical agglomerative clustering where both cluster dendograms are shown alongside a heatmap that represents the feature values as colors. 

Let's consider a gene expression dataset with 63 data points that represent samples that belong to one of two classes: breast cancer or colon cancer. For each sample the expression value of 109 selected genes were measured, these constitute the features. To facilitate the interpretation of the result of a hierarchical clustering it is useful provide identifiers for both the rows and the columns in a data set. The identifiers for the columns are the feature names provides in the first line (the header) a the data set. For the row identifiers we have to tell pandas which row tu use for this by setting the `index_col` function argument of `read_csv()`. This way the hierarchical clustering tool knows that this is a row identifier column that should be used as a data point feature:

In [ ]:
dataset_genes = pd.read_csv("AP_Breast_Colon.csv",index_col=0)
tissue = dataset_genes.pop('Tissue')
print(dataset_genes.shape)

In [ ]:
tissue.value_counts()

In [ ]:
dataset_genes.head()

We first take a look at the individual features:

In [ ]:
plt.figure(figsize=(14,4))
dataset_genes.boxplot()
plt.show()

We clearly need to normalize or standardize these features:

In [ ]:
from sklearn.preprocessing import StandardScaler

dataset_genes_cols = dataset_genes.columns.values
scaler = StandardScaler()
scaler.fit(dataset_genes)
dataset_genes_norm = pd.DataFrame(scaler.transform(dataset_genes),columns=dataset_genes_cols)
plt.figure(figsize=(14,4))
dataset_genes_norm.boxplot()
plt.show()

We can then use the `clustermap()` function to compute the hierarchical clustering. As we know the classes of the data points (remark that these were not used to perform the clustering) in the data set we can plot these as well (blue indicates colon cancer, yellow indicates breast cancer):

In [ ]:
class_colors = ['y' if x == 'Breast' else 'b' for x in tissue]
cg = sns.clustermap(dataset_genes_norm.T,method="complete",col_colors=class_colors,figsize=(14,30))
plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.show()